In [20]:
import pdftotext

with open("gender_trouble.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

    for i in range(len(pdf)):
        page = pdf[i]
        pdf_page_num = i + 1

        print(pdf_page_num)
        if len(page.strip()) != 0:
            print("PAGE", page.strip()[-5:])


        # write code that adds the printed page number and the actual page number in the pdf


is
xxxi


